In [31]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


In [32]:
wine_data = pd.read_csv('https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv')
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [33]:
def define_outputs(x):
    if x >= 7: 
        x = 1
    else:
        x = 0
    return x

In [34]:
#normalizing input features

wine_data_inputs = (wine_data-wine_data.mean())/wine_data.std()

wine_data_outputs = wine_data['quality']

wine_data_outputs = wine_data_outputs.apply(define_outputs)
wine_data_outputs

0       0
1       0
2       0
3       0
4       0
       ..
1594    0
1595    0
1596    0
1597    0
1598    0
Name: quality, Length: 1599, dtype: int64

In [35]:
X = wine_data_inputs.drop('quality', axis = 1)
y = wine_data_outputs

In [36]:
X = X.to_numpy()
y = y.to_numpy()

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42, shuffle = True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

y_train = y_train
y_test = y_test

In [38]:
#defining model

class model(nn.Module):
    def __init__(self, in_features = 11, h1_output = 32, h2_output = 16, h3_output = 16, out_features = 2):
        super().__init__()
        self.h1 = nn.Linear(in_features, h1_output)
        self.h2 = nn.Linear(h1_output, h2_output)
        self.h3 = nn.Linear(h2_output, h3_output)
        self.out = nn.Linear(h2_output, out_features)
    
    def forward(self, X):
        X = F.relu(self.h1(X))
        X = F.relu(self.h2(X))
        X = F.relu(self.h3(X))
        X = F.sigmoid(self.out(X))

        return X

In [39]:
model = model()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .001)

In [40]:
#training loop

epochs = 500
losses = []

for epoch in range(epochs):
    y_pred = model.forward(X_train)

    loss = loss_fn(y_pred, y_train)

    losses.append(loss.detach().numpy())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if(epoch%10 == 0):
        print(f'Epoch: {epoch}, loss: {loss.detach().numpy()}')


Epoch: 0, loss: 0.6815981268882751
Epoch: 10, loss: 0.6738459467887878
Epoch: 20, loss: 0.6660715937614441
Epoch: 30, loss: 0.6556709408760071
Epoch: 40, loss: 0.6390843391418457
Epoch: 50, loss: 0.6134943962097168
Epoch: 60, loss: 0.5779060125350952
Epoch: 70, loss: 0.5358601808547974
Epoch: 80, loss: 0.4963547885417938
Epoch: 90, loss: 0.46855631470680237
Epoch: 100, loss: 0.45318570733070374
Epoch: 110, loss: 0.445034384727478
Epoch: 120, loss: 0.4398004412651062
Epoch: 130, loss: 0.4356172978878021
Epoch: 140, loss: 0.43202072381973267
Epoch: 150, loss: 0.4289635717868805
Epoch: 160, loss: 0.4262934923171997
Epoch: 170, loss: 0.4239608347415924
Epoch: 180, loss: 0.42193126678466797
Epoch: 190, loss: 0.42014381289482117
Epoch: 200, loss: 0.4185466468334198
Epoch: 210, loss: 0.41707566380500793
Epoch: 220, loss: 0.4157349169254303
Epoch: 230, loss: 0.41442686319351196
Epoch: 240, loss: 0.4131574332714081
Epoch: 250, loss: 0.4118172228336334
Epoch: 260, loss: 0.4103674292564392
Epoch:

In [44]:
#testing data accuracy

with torch.no_grad():
    correct = 0
    y_test_pred = model.forward(X_test)
    for i in range(y_test_pred.shape[0]):
        if(y_test_pred[i].argmax() == y_test[i]):
            correct += 1

print(f'Accuracy: {correct}/{y_test_pred.shape[0]}')

Accuracy: 282/320


In [42]:
#training data accuracy

with torch.no_grad():
    correct = 0
    y_train_pred = model.forward(X_train)
    for i in range(y_train_pred.shape[0]):
        if(y_train_pred[i].argmax() == y_train[i]):
            correct += 1

print(f'Accuracy: {correct}/{y_train_pred.shape[0]}')

Accuracy: 1194/1279
